# Useful Notebook: Report Testing Data Prediction Probabilities and Illustrate Accessing Evaluation Metrics
**This notebook will (1) show users how to access all model evaluation metrics from internal pickle files, and (2) generate model (class 1) prediction probabilities for instances of the respective testing dataset.**

*This notebook is designed to run after having run STREAMLINE (at least phases 1-6) and will use the files from a specific STREAMLINE experiment folder, as well as save new output files to that same folder.*

***
## Notebook Details
STREAMLINE outputs pickled objects with (1) all the metric results, (2) elements needed to build the ROC and PRC plots, as well as (3) the prediction probabilities on the testing data across all datasets, algorithm models, and CV dataset partitions. 

This notebook illustrates how the user can access the pickled metric information saved as a list object. 

It includes (1) grabbing and calculating all average metric scores over the CV partitions, (2) grabbing the elements needed to build the average ROC plot, (3) grabbing the elementes needed to build the average PRC plot, (4) grabbing and reporting average model feature importance scores, and (5) grabbing and reporting the model testing prediction probabilities for each instance of the dataset. 

When run, this last item will generate a new folder (`prediction_probas`) in the pipeline's output experiment folder in the `model_evaluation` folder for each dataset. Here the class 1 prediction probabilities are reported as a `.csv` file for each algorithm and CV partition pair. In these files is the instance's true outcome value, the unique instance ID, and the predicted probability of the instance being class 1 (i.e. which typically encodes cases or the less frequent class). 
 

***
## Notebook Run Parameters
* This notbook has been set up to run 'as-is' on the experiment folder generated when running the demo of STREAMLINE in any mode (if no run parameters were changed). 
* If you have run STREAMLINE on different target data or saved the experiment to some other folder outside of STREAMLINE, you need to edit `experiment_path` below to point to the respective experiment folder.

In [1]:
experiment_path = "../DemoOutput/demo_experiment" # path the target experiment folder 
target_data_list = None # None if user wants to generate output for all analyzed target datasets, otherwise provide a (str) list of target dataset names to run
algorithms = [] # use empty list if user wishes re-evaluate all modeling algorithms that were run in pipeline, otherwise specify a (str) list of algorithm identifiers.

***
## Housekeeping
### Import Packages

In [2]:
import os
import pandas as pd
import pickle
import numpy as np
from statistics import mean
from scipy import interp,stats
import warnings
warnings.filterwarnings('ignore')

# Jupyter Notebook Hack: This code ensures that the results of multiple commands within a given cell are all displayed, rather than just the last. 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Automatically Detect Dataset Names

In [3]:
# Get dataset paths for all completed dataset analyses in experiment folder
datasets = os.listdir(experiment_path)

# Name of experiment folder
experiment_name = experiment_path.split('/')[-1] 

datasets = os.listdir(experiment_path)
remove_list = ['.DS_Store', 'metadata.pickle', 'metadata.csv', 'algInfo.pickle',
                'DatasetComparisons', 'jobs', 'jobsCompleted', 'logs',
                'KeyFileCopy', 'dask_logs',
                experiment_name + '_ML_Pipeline_Report.pdf']
for text in remove_list:
    if text in datasets:
        datasets.remove(text)

datasets = sorted(datasets) # ensures consistent ordering of datasets
print("Analyzed Datasets: " + str(datasets))

Analyzed Datasets: ['hcc_data', 'hcc_data_custom']


### Load Other Necessary Parameters

In [4]:
# Unpickle metadata from previous phase
file = open(experiment_path + '/' + "metadata.pickle", 'rb')
metadata = pickle.load(file)
file.close()
# Load variables specified earlier in the pipeline from metadata
class_label = metadata['Class Label']
instance_label = metadata['Instance Label']
cv_partitions = int(metadata['CV Partitions'])

# Unpickle algorithm information from previous phase
file = open(experiment_path + '/' + "algInfo.pickle", 'rb')
algInfo = pickle.load(file)
file.close()
algorithms = []
abbrev = {}
for key in algInfo:
    if algInfo[key][0]: # If that algorithm was used
        algorithms.append(key)
        abbrev[key] = (algInfo[key][1])

print("Algorithms Ran: " + str(algorithms))

Algorithms Ran: ['Decision Tree', 'Logistic Regression', 'Naive Bayes']


***
## From Pickle: Extract Metric List and Cacluate CV Averages

In [5]:
def print_results(algorithm, full_path):
    # Define evaluation stats variable lists
        s_bac = [] # balanced accuracies
        s_ac = [] # standard accuracies
        s_f1 = [] # F1 scores
        s_re = [] # recall values
        s_sp = [] # specificities
        s_pr = [] # precision values
        s_tp = [] # true positives
        s_tn = [] # true negatives
        s_fp = [] # false positives
        s_fn = [] # false negatives
        s_npv = [] # negative predictive values
        s_lrp = [] # likelihood ratio positive values
        s_lrm = [] # likelihood ratio negative values
        
        aucs = [] #areas under ROC curve
        praucs = [] #area under PRC curve
        aveprecs = [] #average precisions for PRC
        
        for cv_count in range(0, cv_partitions): #loop through cv's
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            metric_list = results[0] #First item in pickled list is the metric list (set of standard classification metrics)
            roc_auc = results[3] #Fourth item is the ROC AUC
            prec_rec_auc = results[6] #Seventh item is the PRC AUC
            ave_prec = results[7] #Eighth item is the average precision of PRC
            
            #Separate metrics from metricList
            s_bac.append(metric_list[0])
            s_ac.append(metric_list[1])
            s_f1.append(metric_list[2])
            s_re.append(metric_list[3])
            s_sp.append(metric_list[4])
            s_pr.append(metric_list[5])
            s_tp.append(metric_list[6])
            s_tn.append(metric_list[7])
            s_fp.append(metric_list[8])
            s_fn.append(metric_list[9])
            s_npv.append(metric_list[10])
            s_lrp.append(metric_list[11])
            s_lrm.append(metric_list[12])
            
            aucs.append(roc_auc)
            praucs.append(prec_rec_auc)
            aveprecs.append(ave_prec)
            
        results = {'Balanced Accuracy': mean(s_bac), 'Accuracy': mean(s_ac), 
                   'F1_Score': mean(s_f1), 'Sensitivity (Recall)': mean(s_re), 
                   'Specificity': mean(s_sp),'Precision (PPV)': mean(s_pr), 
                   'TP': mean(s_tp), 'TN': mean(s_tn), 'FP': mean(s_fp), 
                   'FN': mean(s_fn), 'NPV': mean(s_npv), 'LR+': mean(s_lrp), 
                   'LR-': mean(s_lrm), 'ROC_AUC': mean(aucs),'PRC_AUC': mean(praucs), 
                   'PRC_APS': mean(aveprecs)}
        print(results)

In [6]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)

for each in datasets: 
    print("---------------------------------------")
    print("Dataset: "+str(each))
    print("---------------------------------------")
    full_path = experiment_path + '/' + each
    for algorithm in algorithms: #loop through algorithms
        print("Algorithm: "+str(algorithm))
        print_results(algorithm, full_path)

---------------------------------------
Dataset: hcc_data
---------------------------------------
Algorithm: Decision Tree
{'Balanced Accuracy': 0.6360877684407096, 'Accuracy': 0.6363636363636364, 'F1_Score': 0.5711193632484838, 'Sensitivity (Recall)': 0.6349206349206349, 'Specificity': 0.6372549019607843, 'Precision (PPV)': 0.5466272633455296, 'TP': 13, 'TN': 21, 'FP': 12, 'FN': 7, 'NPV': 0.7429402382962754, 'LR+': 2.073922902494331, 'LR-': 0.561180882609454, 'ROC_AUC': 0.7166199813258637, 'PRC_AUC': 0.6218233079232455, 'PRC_APS': 0.5504193900964013}
Algorithm: Logistic Regression
{'Balanced Accuracy': 0.6911764705882353, 'Accuracy': 0.696969696969697, 'F1_Score': 0.6273602216979366, 'Sensitivity (Recall)': 0.6666666666666666, 'Specificity': 0.7156862745098039, 'Precision (PPV)': 0.5932323232323232, 'TP': 14, 'TN': 24, 'FP': 9, 'FN': 7, 'NPV': 0.7762463343108504, 'LR+': 2.3868686868686866, 'LR-': 0.46695776043602133, 'ROC_AUC': 0.7726423902894491, 'PRC_AUC': 0.666547261956175, 'PRC_AP

## From Pickle: Extract list of true and false positive rates for constructing ROC

In [7]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)

for each in datasets: 
    print("---------------------------------------")
    print("Dataset: "+str(each))
    print("---------------------------------------")
    full_path = experiment_path+ '/' + each
    for algorithm in algorithms: #loop through algorithms
        print("Algorithm: "+str(algorithm))
        # Define evaluation stats variable lists
        tprs = [] # true postitive rates
        mean_fpr = np.linspace(0, 1, 100) # used to plot all CVs in single ROC plot
        
        for cv_count in range(0, cv_partitions): #loop through cv's
            # Load pickled metric file for given algorithm and cv =
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            fpr = results[1]
            tpr = results[2]

            tprs.append(interp(mean_fpr, fpr, tpr))
            tprs[-1][0] = 0.0

        results = {'tprs': np.mean(tprs, axis=0)}
        
        print(results)
        #print('fprs: '+str(mean_fpr))

---------------------------------------
Dataset: hcc_data
---------------------------------------
Algorithm: Decision Tree
{'tprs': array([0.        , 0.03270803, 0.06541607, 0.0962001 , 0.10710277,
       0.11800545, 0.13051146, 0.15049971, 0.17048795, 0.22222222,
       0.24221046, 0.26219871, 0.28218695, 0.30217519, 0.32216343,
       0.34295334, 0.3647587 , 0.38656405, 0.42231842, 0.44048955,
       0.45866068, 0.47683181, 0.49500294, 0.51317407, 0.5294212 ,
       0.54486666, 0.56031212, 0.57864358, 0.59772326, 0.61680295,
       0.64309764, 0.67035434, 0.69761103, 0.70357143, 0.70888648,
       0.71484287, 0.73106061, 0.74727834, 0.76189274, 0.76720779,
       0.77252285, 0.7778379 , 0.78315296, 0.78846801, 0.79378307,
       0.79909812, 0.80441318, 0.81085057, 0.81889129, 0.82693202,
       0.83497274, 0.84301347, 0.85105419, 0.857692  , 0.86334776,
       0.86900353, 0.8737681 , 0.87810646, 0.88244482, 0.88678318,
       0.89112153, 0.89545989, 0.89843911, 0.901184  , 0.9039288

## From Pickle: Extract list of precision and recall values for constructing PRC

In [8]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)

for each in datasets: 
    print("---------------------------------------")
    print("Dataset: "+str(each))
    print("---------------------------------------")
    full_path = experiment_path + '/' + each
    for algorithm in algorithms: #loop through algorithms
        print("Algorithm: "+str(algorithm))
        # Define evaluation stats variable lists
        precs = [] # true postitive rates
        mean_recall = np.linspace(0, 1, 100) # used to plot all CVs in single PRC plot
        
        for cv_count in range(0, cv_partitions): #loop through cv's
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            prec = results[4]
            recall = results[5]

            precs.append(interp(mean_recall, recall, prec))

        results = {'precs': np.mean(precs, axis=0)}

        print(results)
        #print('recall: '+str(mean_recall))

---------------------------------------
Dataset: hcc_data
---------------------------------------
Algorithm: Decision Tree
{'precs': array([1.        , 0.98146618, 0.96293235, 0.94439853, 0.92586471,
       0.90733088, 0.88879706, 0.87026324, 0.85172942, 0.83319559,
       0.81466177, 0.79612795, 0.77759412, 0.7590603 , 0.74052648,
       0.73588154, 0.73355142, 0.7312213 , 0.72889118, 0.68257357,
       0.68344211, 0.68431064, 0.68517917, 0.68604771, 0.68448118,
       0.67966789, 0.67485461, 0.67004132, 0.66522804, 0.66041475,
       0.65560147, 0.65078818, 0.6459749 , 0.64116162, 0.63634833,
       0.63153505, 0.62672176, 0.62190848, 0.61803315, 0.61650269,
       0.61497223, 0.61344178, 0.61191132, 0.60907844, 0.60526874,
       0.60145904, 0.59764935, 0.59383965, 0.59002995, 0.58622025,
       0.58241055, 0.57860085, 0.57485278, 0.57147443, 0.56809609,
       0.56471774, 0.56133939, 0.55933309, 0.55915617, 0.55897925,
       0.55880234, 0.55862542, 0.5584485 , 0.55827158, 0.558094

## From Pickle: Extract Average Model Feature Importance Estimates (Over CVs)

In [9]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)
print("Dataset: " + str(datasets))

for each in datasets: 
    print("---------------------------------------")
    print("Dataset: "+str(each))
    print("---------------------------------------")
    full_path = experiment_path + '/' + each
    original_headers = pd.read_csv(full_path + "/exploratory/ProcessedFeatureNames.csv", sep=',').columns.values.tolist() # Get Original Headers
    for algorithm in algorithms: #loop through algorithms
        print("Algorithm: "+str(algorithm))
        # Define evaluation stats variable lists
        FI_ave = [0] * len(original_headers)  # used to save average FI scores over all cvs. (all original features in dataset prior to feature selection included)
        
        for cv_count in range(0, cv_partitions): # loop through cv's
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            fi = results[8]
            
            # Format feature importance scores as list (takes into account that all features are not in each CV partition)
            tempList = []
            j = 0
            headers = pd.read_csv(full_path + '/CVDatasets/' + each + '_CV_' + str(cv_count) + '_Test.csv').columns.values.tolist()
            if instance_label != None: 
                headers.remove(instance_label)
            headers.remove(class_label)
            for feature in original_headers:
                if feature in headers:  # Check if current feature from original dataset was in the partition
                    # Deal with features not being in original order (find index of current feature list.index()
                    f_index = headers.index(feature)
                    FI_ave[j] += fi[f_index]
                j += 1
            
        #Turn FI sums into averages
        for i in range(0, len(FI_ave)):
            FI_ave[i] = FI_ave[i] / float(cv_partitions)

        fi_dict = {}
        for key in original_headers:
            for value in FI_ave:
                fi_dict[key] = value
                FI_ave.remove(value)
                break  
                
        print(fi_dict)

Dataset: ['hcc_data', 'hcc_data_custom']
---------------------------------------
Dataset: hcc_data
---------------------------------------
Algorithm: Decision Tree
{'Gender': 0.0, 'Symptoms': 0.026937441643323973, 'Alcohol': 0.0, 'Hepatitis B Surface Antigen': 0.0, 'Hepatitis B e Antigen': 0.0, 'Hepatitis B Core Antibody': 0.0, 'Hepatitis C Virus Antibody': 0.0, 'Cirrhosis': 0.0, 'Endemic Countries': 0.0, 'Smoking': 0.0, 'Diabetes': 0.0, 'Obesity': 0.0, 'Hemochromatosis': 0.0, 'Arterial Hypertension': 0.0, 'Chronic Renal Insufficiency': 0.0, 'Human Immunodeficiency Virus': 0.0, 'Nonalcoholic Steatohepatitis': 0.0, 'Esophageal Varices': 0.0, 'Splenomegaly': 0.0, 'Portal Hypertension': 0.0, 'Portal Vein Thrombosis': 0.0, 'Liver Metastasis': 0.02169701213818859, 'Radiological Hallmark': 0.0, 'Age at diagnosis': 0.0, 'Grams of Alcohol per day': 0.0, 'Packs of cigarets per year': 0.0, 'Performance Status*': 0.01768207282913162, 'Encephalopathy degree*': 0.0, 'Ascites degree*': 0.04682539682

## Extract and Output Testing Data Prediction Probabilities 

In [10]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)

for each in datasets: 
    print("---------------------------------------")
    print("Dataset: "+str(each))
    print("---------------------------------------")

    full_path = experiment_path + '/' + each
    
    # Make folder in experiment folder/datafolder to store all prediction probabilities per algorithm/CV combination
    if not os.path.exists(full_path + '/model_evaluation/prediction_probas'):
        os.mkdir(full_path + '/model_evaluation/prediction_probas')
        
    original_headers = pd.read_csv(full_path + "/exploratory/OriginalFeatureNames.csv", sep=',').columns.values.tolist() #Get Original Headers
    for algorithm in algorithms: #loop through algorithms
        print("Algorithm: "+str(algorithm))

        for cv_count in range(0,cv_partitions): #loop through cv's
            print("CV: "+str(cv_count))
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Load associated testing dataset
            test_data = pd.read_csv(full_path + '/CVDatasets/'+each+'_CV_' + str(cv_count) + '_Test.csv')
            probas_summary = test_data[[class_label,instance_label]]

            #Separate pickled results
            probas_ = results[9]
            print(probas_[:,1])
            probas_summary['1_prob'] = probas_[:,1]
            file_name = full_path + '/model_evaluation/prediction_probas/' + algorithm + '_CV_'+str(cv_count) + '_class1_probas.csv'
            probas_summary.to_csv(file_name, index=False)

---------------------------------------
Dataset: hcc_data
---------------------------------------
Algorithm: Decision Tree
CV: 0
[0.94182825 0.94182825 0.         0.72961373 0.         0.
 0.76404494 0.89005236 0.94182825 0.80188679 0.24460432 0.5483871
 0.35051546 0.94182825 0.72961373 0.35051546 0.         0.24460432
 0.80188679 0.80188679 0.35051546 0.24460432 0.72961373 0.5483871
 0.24460432 0.24460432 0.         0.24460432 0.24460432 0.
 0.         0.72961373 0.         0.24460432 0.         0.
 0.         0.94182825 0.35051546 0.         0.35051546 0.80188679
 0.         0.5483871  0.24460432 0.         0.80188679 0.
 0.24460432 0.         0.24460432 0.         0.80188679 0.94182825
 0.89005236]
CV: 1
[0.51276102 0.11333333 0.11333333 0.51276102 0.80804388 0.51276102
 0.80804388 0.80804388 0.11333333 0.80804388 0.11333333 0.80804388
 0.11333333 0.80804388 0.51276102 0.51276102 0.11333333 0.11333333
 0.11333333 0.80804388 0.51276102 0.80804388 0.80804388 0.80804388
 0.51276102 0.5

[5.60264574e-08 2.12628928e-07 1.08158631e-05 2.59936194e-04
 3.69439090e-11 6.27628652e-07 2.50494900e-08 4.20626389e-10
 3.62669682e-04 4.55084516e-02 2.89475802e-10 3.48454463e-07
 1.10456395e-08 1.13797762e-07 3.60938246e-07 4.72493991e-04
 2.29007400e-09 2.54777131e-08 3.70482359e-03 9.99998950e-01
 3.53354608e-07 4.60723744e-10 6.62014704e-03 1.02258410e-04
 1.68531448e-03 8.98443382e-01 2.78982677e-05 2.15818565e-03
 1.10531967e-08 6.10181983e-08 3.70596068e-09 4.30043579e-07
 1.14537000e-02 4.94031634e-07 1.00000000e+00 9.99987675e-01
 1.42679847e-03 1.53536674e-06 9.98547729e-01 4.96955438e-08
 6.60972762e-04 1.54774868e-10 2.96702464e-05 1.00000000e+00
 1.35458113e-01 2.04265858e-09 3.39877786e-08 5.28572734e-09
 8.98911683e-08 3.12254190e-11 2.25580548e-10 6.77701730e-06
 8.20141465e-08 5.23784021e-04 1.00000000e+00]
